In [1]:
# Imports
import argparse
import os
import sys
import time
import glob
import re

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
from _datetime import datetime
import scipy.stats as stats
import matplotlib.animation as animation
from matplotlib import rcParams
from PIL import Image
from sklearn.utils import resample
from scipy.stats import pearsonr
import scipy.stats as stats
import scipy

In [2]:
# Local imports
sys.path.append("/home/users/benhutch/skill-maps")
import dictionaries as dicts

sys.path.append("/home/users/benhutch/skill-maps/python")
import functions as fnc

In [3]:
# Imports of historical functions
#/home/users/benhutch/skill-maps-differences/functions.py
import os

path = "/home/users/benhutch/skill-maps-differences"
print(os.listdir(path))

# append path and import
sys.path.append(path)
import functions_diff as hist_fnc

# List the functions in the module
print(dir(hist_fnc))

['.git', '.gitignore', 'README.md', 'functions_diff.py', 'dictionaries.py', 'plotting.ipynb', '__pycache__', 'plotting_functions.py', '.ipynb_checkpoints', 'skill_maps_functions.py']
['CDOException', 'CDO_PY_VERSION', 'Cdo', 'CdoTempfileStore', 'Image', 'StringIO', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'animation', 'argparse', 'auto_doc', 'calculate_annual_mean_anomalies_obs', 'calculate_anomalies', 'calculate_correlation_coefficient', 'calculate_correlations', 'calculate_correlations_diff', 'calculate_dcpp_data_ws', 'calculate_historical_data_ws', 'calculate_spatial_correlations', 'calculate_spatial_correlations_bootstrap', 'calculate_spatial_correlations_bootstrap_diff', 'calculate_spatial_correlations_diff', 'calculate_wind_speed', 'ccrs', 'cdo', 'constrain_years_init_uninit', 'constrain_years_processed_hist', 'da', 'dask', 'datetime', 'dd', 'dic', 'extract_ensemble_members', 'extract_historical_data', 'getCdoVersio

In [4]:
# Set up the parameters
# Test run with psl dcppA-hindcast raw data and historical data
variable = "psl"
obs_var_name = "psl"
region = "global"
region_grid = dicts.gridspec_global
forecast_range = "2-9"
season = "DJFM"
model_season = "DJFM"
start_year = "1960"
end_year = "2022"

# Set up the paths
observations_path = dicts.obs
dcpp_models = dicts.models
historical_models = dicts.psl_models_historical
base_dir = dicts.base_dir
base_dir_historical = dicts.base_dir_historical
output_dir = dicts.plots_dir
save_dir = dicts.save_dir

In [5]:
%tb
# Process the observations
obs = fnc.process_observations(variable, region, region_grid, forecast_range, season,
                                observations_path, obs_var_name, plev=None)

Gridspec file: /home/users/benhutch/gridspec/gridspec-global.txt
Variable is not ua or va, creating new file name
File already exists
Loading ERA5 data


No traceback available to show.


In [6]:
# Load the historical data
hist_data = hist_fnc.load_processed_historical_data(base_dir_historical, historical_models,
                                                    variable, region, forecast_range, season)

# Extract the data for the variable from the historical data
hist_data, _ = hist_fnc.extract_historical_data(hist_data, variable)

processing model:  BCC-CSM2-MR
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/psl/BCC-CSM2-MR/global/years_2-9/DJFM/outputs/processed/*.nc
processing model:  MPI-ESM1-2-HR
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/psl/MPI-ESM1-2-HR/global/years_2-9/DJFM/outputs/processed/*.nc
processing model:  CanESM5
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/psl/CanESM5/global/years_2-9/DJFM/outputs/processed/*.nc
processing model:  CMCC-CM2-SR5
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/psl/CMCC-CM2-SR5/global/years_2-9/DJFM/outputs/processed/*.nc
processing model:  HadGEM3-GC31-MM
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/psl/HadGEM3-GC31-MM/global/years_2-9/DJFM/outputs/processed/*.nc
processing model:  EC-Earth3
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/psl/EC-Earth3/global/years_2-9/DJFM/outputs/processed/*.nc
processing model:  M

TypeError: 'NoneType' object is not iterable

In [ ]:
# Load the DCPP data
dcpp_data = fnc.load_data(base_dir, dcpp_models, variable, region, 
                            forecast_range, season, level=None)

# Extract the data for the variable from the DCPP data
dcpp_data, _ = fnc.process_data(dcpp_data, variable)

In [ ]:
# Now we want to make sure that the time period 
# is the same for all of the data
# and extract the data into arrays
